In [21]:
from database import *

def ticks_to_kbars(ticks, interval='1Min'):

    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

In [22]:
import talib

def get_technical_indicator(kbars):
    
    kbars['rsi'] = talib.RSI(kbars.close, timeperiod=14)
    kbars['cci'] = talib.CCI(kbars.high, kbars.low, kbars.close, timeperiod=14)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=5, slowperiod=34, signalperiod=5)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    kbars['slowk'], kbars['slowd'] = talib.STOCH(kbars.high,
                        kbars.low,
                        kbars.close,
                        fastk_period=9,
                        slowk_period=3,
                        slowk_matype=0,
                        slowd_period=3,
                        slowd_matype=0)

    upper, middle, lower = talib.BBANDS(kbars.close, 
                                        timeperiod=20, 
                                        nbdevup=2.1, 
                                        nbdevdn=2.1, 
                                        # Moving average type: simple moving average here
                                        matype=0)
    kbars['upper'] = upper
    kbars['middle'] = middle
    kbars['lower'] = lower    
    
    return kbars

In [23]:
import json

with open('config.json') as f:
  config = json.load(f)

api = sj.Shioaji()
api.login(
    person_id = config['account'],
    passwd= config['password'],
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done."))

# api.logout()

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.


In [24]:
code = '4935'

tw_calendar = get_calendar('XTAI')

date = pd.to_datetime('2020-3-26')
prev_trading_date = tw_calendar.previous_close(date).date()

ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
kbars = ticks_to_kbars(ticks, '5Min')
kbars = get_technical_indicator(kbars)
kbars = kbars[pd.to_datetime(prev_trading_date).replace(hour=13, minute=30):]
# kbars = kbars[date:]

kbars

<SecurityType.Stock: 'STK'> fetch done.


,open,close,high,low,volume,rsi,cci,macd,macdsignal,macdhist,slowk,slowd,upper,middle,lower
ts,,,,,,,,,,,,,,,
2020-03-25 13:30:00,98.3,98.3,98.3,98.3,7,100.000000,66.666667,0.023095,0.026262,-0.003167,4.736952e-15,1.105289e-14,98.300000,98.300,98.300000
2020-03-26 09:00:00,99.5,97.4,99.5,97.0,662,10.047240,-466.666667,-0.226796,-0.058091,-0.168705,5.333333e+00,1.777778e+00,98.666916,98.255,97.843084
2020-03-26 09:05:00,97.4,95.3,98.0,95.0,486,3.081643,-433.918129,-0.959551,-0.358577,-0.600973,7.555556e+00,4.296296e+00,99.517592,98.105,96.692408
2020-03-26 09:10:00,95.4,96.4,97.0,95.2,247,30.328878,-226.771310,-1.098053,-0.605069,-0.492983,1.792593e+01,1.027160e+01,99.618797,98.010,96.401203
2020-03-26 09:15:00,96.7,97.5,97.5,96.3,175,46.520444,-86.013072,-0.860386,-0.690175,-0.170211,3.111111e+01,1.886420e+01,99.588636,97.970,96.351364
2020-03-26 09:20:00,97.5,98.1,98.8,97.4,592,52.944245,25.617808,-0.528893,-0.636414,0.107521,5.185185e+01,3.362963e+01,99.572220,97.960,96.347780
2020-03-26 09:25:00,98.2,97.6,98.7,97.5,305,47.792360,10.226836,-0.448547,-0.573792,0.125245,6.074074e+01,4.790123e+01,99.536502,97.925,96.313498
2020-03-26 09:30:00,97.6,99.9,101.0,97.6,710,64.773304,167.209555,0.245738,-0.300615,0.546353,6.944444e+01,6.067901e+01,99.848311,98.005,96.161689
2020-03-26 09:35:00,99.9,102.5,103.0,99.8,981,74.765364,296.184063,1.395560,0.264776,1.130784,7.773148e+01,6.930556e+01,100.978934,98.215,95.451066


<SecurityType.Option: 'OPT'> fetch done.


In [5]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=5)
        print(' start current_time = {}'.format(current_time))
        current_price = kbars['close'][ts]
        print('current_price = {}'.format(current_price))
#         macdhist = kbars.iloc[ts]['macdhist']
#         print('macdhist = {}'.format(macdhist))
#         cci = kbars.iloc[ts]['cci']
#         print('cci = {}'.format(cci))
#         print('end')
        
        if (
#             kbars.iloc[ts - 3:ts]['rsi'].min() < 30 and
#             kbars.iloc[ts - 1]['macdhist'] < 0 and
            current_time < date.replace(hour=13, minute=20, second=0) and
            kbars.iloc[ts]['cci'] > -100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
#             kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
#             kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['cci'] > 100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time >= date.replace(hour=13, minute=20, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

<SecurityType.Option: 'OPT'> fetch done.


In [6]:
day_trading_backtest('2456', pd.to_datetime('2021/3/2'), connection, api)

 start current_time = 2021-03-02 09:05:00
current_price = 114.0
[2021-03-02 09:05:00] buy 2456 at 114.0
 start current_time = 2021-03-02 09:06:00
current_price = 114.0
[2021-03-02 09:06:00] sell 2456 at 114.0


,成交日期,股票代號,買進時間,買進價格,買進股數,買進金額,賣出時間,賣出價格,賣出股數,賣出金額
0,2021-03-02,2456,09:05:00,114.0,1000,114000.0,09:06:00,114.0,1000,114000.0
